<a href="https://colab.research.google.com/github/ELHADRINOUHAYLA/MNISTARABIC/blob/main/letters_rename_resize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import ImageChops,Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2


In [ ]:
def trim(image):
    image=Image.fromarray(image)
    bg = Image.new(image.mode, image.size, image.getpixel((0,0)))
    diff = ImageChops.difference(image, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return np.array(image.crop(bbox))
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized
def cleaned_image(image_path):
    image=cv2.imread(image_path)
    bg=np.zeros((32,32))
    image=trim(image)
    image=image_resize(image,height=28)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh, image_binary = cv2.threshold(image_gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    hh, ww = bg.shape
    h, w = image_binary.shape
    yoff = round((hh-h)/2)
    xoff = round((ww-w)/2)
    if xoff<=0:
        image=image_resize(image,height=28,width=28)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh, image_binary = cv2.threshold(image_gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        h, w = image_binary.shape
        yoff = round((hh-h)/2)
        xoff = round((ww-w)/2)
    result = bg.copy()
    result[yoff:yoff+h, xoff:xoff+w] = image_binary
    return result

In [ ]:
import os
import glob
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)
def create_dataset(input,output,name):
    for char in glob.iglob(input+'/*'):
        char_name=char.split('\\')[-1].lower()
        for type in glob.iglob(char+'/*'):
            type_name=type.split('\\')[-1].lower()
            i=0
            for image_path in glob.iglob(type+'/*'):
                create_directory(output)
                create_directory(output+'/'+char_name)
                create_directory(output+'/'+char_name+'/'+type_name)
                ext=image_path.split('\\')[-1].split('.')[1].lower()
                cv2.imwrite(output+'/'+char_name+'/'+type_name+'/'+name+'_'+str(i)+"_"+type_name+"."+ext,cleaned_image(image_path))
                i+=1

In [ ]:
create_dataset(input='data/letters',output='letter',name='hicham_achahboun')